# Notebook de préparation des données et fusion des données des Inscrits et des Résultats de UCAD

**Contexte**

Notre étude porte sur :

### *l’analyse de l'impact des dernières réformes du baccalauréat au Sénégal sur le taux de réussite au Bac et l’insertion universitaire*.

Afin d’étudier les cohortes d’étudiants impactés par les dernières réformes du baccalauréat, il est nécessaire d’avoir une vue sur leur réussite à l’UCAD (à travers les données de résultats). Cependant, comme ces données ne contiennent ni l’année d’obtention du Bac, ni la série obtenue, il est indispensable de les fusionner avec les données d’inscription, qui, elles, contiennent ces informations. La fusion se fait à l’aide de la clé commune : **le numéro d’étudiant** 

## Importation des packages nécessaires 

In [1]:
import warnings
warnings.filterwarnings('ignore') # ignore warnings from sklearn

In [2]:
import pandas as pd # pour la manipulation de données
import os # pour les opérations sur les chemins de fichiers
from tqdm import tqdm # Affichage de barres de progression

## Importation des Données

### Données des Inscrits

In [3]:
file_path = os.path.join(os.getcwd(), "data_csv_inscrits") # Chemin vers le dossier contenant les fichiers CSV
file = os.listdir(file_path) # Liste des fichiers dans le dossier

print("liste des fichiers csv des inscrits à l'UCAD par année :")
for f in file:
    print(f)

liste des fichiers csv des inscrits à l'UCAD par année :
2002.csv
2003.csv
2004.csv
2005.csv
2006.csv
2007.csv
2008.csv
2009.csv
2010.csv
2011.csv
2012.csv
2013.csv
2014.csv
2015.csv
2016.csv
2017.csv
2018.csv
2019.csv
2020.csv
2021.csv
2022.csv
2023.csv
2024.csv
2025.csv


### Importation des données des Inscrits de 2011 à 2024 pour la Fusion

#### Remarque :

Comme les données de résultats commencent à partir de l’année universitaire 2010-2011, la fusion ne prendra en compte que les inscriptions à partir de 2011, correspondant à cette année universitaire.

In [4]:
# Liste des années
year = [str(a) for a in range(2011, 2024+1)]
data = {}

# Pour stocker les résultats
success_files = []
failed_files = []

# Chargement avec barre de progression
pbr = tqdm(year, desc="Importation des fichiers CSV")
for year in pbr:
    fichier = f"{file_path}/{year}.csv"
    try:
        df = pd.read_csv(fichier, sep=";", encoding='latin-1')  # Charge chaque fichier
        data[year] = df  # Stocke dans le dictionnaire
        success_files.append(year)
        pbr.set_postfix_str(f"{year}.csv chargé ✅")
    except Exception as e:
        failed_files.append((year, str(e)))
        print(f"❌ Erreur inconnue avec {fichier} : {e}")

if success_files:
    print(f"\n✅ Fichiers chargés avec succès ({len(success_files)}):")
    print(", ".join(success_files))

if failed_files:
    print(f"\n❌ Fichiers en erreur ({len(failed_files)}):")
    for year, err in failed_files:
        print(f"  - {year}.csv : {err}")

Importation des fichiers CSV: 100%|██████████| 14/14 [00:05<00:00,  2.59it/s, 2024.csv chargé ✅]


✅ Fichiers chargés avec succès (14):
2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024


### Vérification de l’homogénéité des colonnes dans tous les fichiers

In [5]:
colonnes_reference = set(data["2011"].columns)  # Colonnes de 2011 comme référence
colonnes_identiques = True  # Drapeau pour suivre l'état

for year, df in data.items(): # Parcours des années et vérification des colonnes
    colonnes_actuelles = set(df.columns) # Colonnes actuelles du fichier
    if colonnes_actuelles != colonnes_reference: # Si les colonnes ne correspondent pas
        print(f"⚠️ Les colonnes de {year}.csv sont différentes de celles de 2006.csv")  
        print(f"Colonnes manquantes dans {year}.csv :", colonnes_reference - colonnes_actuelles)
        print(f"Colonnes supplémentaires dans {year}.csv :", colonnes_actuelles - colonnes_reference)
        colonnes_identiques = False 

if colonnes_identiques:
    print("✅ Toutes les colonnes sont identiques dans tous les fichiers.")


✅ Toutes les colonnes sont identiques dans tous les fichiers.


### Données des Résultats

In [6]:
df_resultat = pd.read_csv("resultat_ucad_clen.csv")
print("✅ Importation du fichier resultat_ucad_clen.csv réussie.")

✅ Importation du fichier resultat_ucad_clen.csv réussie.


## Nettoyage et Préparation des données

### Données des Inscrits 

In [7]:
print("Informations sur le DataFrame de 2011 (Tous les fichiers ont les mêmes colonnes) :")
print("--------------------------------------------------")
data['2011'].info()

Informations sur le DataFrame de 2011 (Tous les fichiers ont les mêmes colonnes) :
--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67522 entries, 0 to 67521
Data columns (total 32 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   NUMERO                   67522 non-null  object 
 1   NUMERO_TABLE             0 non-null      float64
 2   INE                      27511 non-null  object 
 3   NUMERO_IDENTITIFICATION  67522 non-null  object 
 4   NOM                      67522 non-null  object 
 5   PRENOM                   67522 non-null  object 
 6   DATE_DE_NAISSANCE        67522 non-null  object 
 7   LIEU_DE_NAISSANCE        67442 non-null  object 
 8   MAIL_INSTITUTIONNEL      33405 non-null  object 
 9   SEXE                     67522 non-null  object 
 10  MENTION_BACC             67151 non-null  object 
 11  ANNEE_BACC               67185 non-null  float64
 

### Creation du DataFram regroupant tous les années

In [8]:
all_data = pd.concat(data.values(), ignore_index=True) # Concatène tous les DataFrames en un seul

print("Affichage du DataFrame combiné:")

all_data[["NUMERO", "MENTION_BACC", "ANNEE_BACC", "SERIE_BACC", "ETABLISSMENT_CODE", "ANNEE_INSCRIPTION"]]

Affichage du DataFrame combiné:


,NUMERO,MENTION_BACC,ANNEE_BACC,SERIE_BACC,ETABLISSMENT_CODE,ANNEE_INSCRIPTION
0,.19950018H,PA,1990.0,NaN,FASTEF,2011
1,.199700AUY,PA,1995.0,D,FMPO,2011
2,.199900QF3,PA,1999.0,L2,FASTEF,2011
3,.199900RLX,PA,1999.0,S2,FMPO,2011
4,.199900RMH,PA,1999.0,S2,FMPO,2011
...,...,...,...,...,...,...
1141115,.20230BQJP,PA,2023.0,L'1,FLSH,2024
1141116,.20230BREG,PA,2023.0,S1,FST,2024
1141117,.20230BXVL,BI,2023.0,S2,FMPO,2024
1141118,.20230BT8N,PA,2023.0,L2,FSJP,2024


In [9]:
print("Informations général du DataFrame :")
print("--------------------------------------------------")
all_data.info()

Informations général du DataFrame :
--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1141120 entries, 0 to 1141119
Data columns (total 32 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   NUMERO                   1141120 non-null  object 
 1   NUMERO_TABLE             116181 non-null   float64
 2   INE                      959252 non-null   object 
 3   NUMERO_IDENTITIFICATION  1141120 non-null  object 
 4   NOM                      1141120 non-null  object 
 5   PRENOM                   1141120 non-null  object 
 6   DATE_DE_NAISSANCE        1141120 non-null  object 
 7   LIEU_DE_NAISSANCE        1140879 non-null  object 
 8   MAIL_INSTITUTIONNEL      1056572 non-null  object 
 9   SEXE                     1141120 non-null  object 
 10  MENTION_BACC             1134817 non-null  object 
 11  ANNEE_BACC               1136174 non-null  float64
 12  PAYS_DE_NAI

#### DataFrame avec seulement les variables les plus importants

In [10]:
# Creation du data frame avec les colonnes importants

list_col_to_drop = ['NUMERO_TABLE', 'INE', 'NUMERO_IDENTITIFICATION', 'NOM', 'PRENOM', 'DATE_DE_NAISSANCE', 'LIEU_DE_NAISSANCE',
                    'MAIL_INSTITUTIONNEL', 'MENTION_BACC', 'PAYS_DE_NAISSANCE', 'REGION_DE_NAISSANCE', 'ETABLISSEMENT', 'SIGLE_NIVEAU_SECTION',
                    'COHORTE', 'NIVEAU', 'DATE_INSCRIPTION', 'NIVEAU LMD', 'ETAT_INSCRIPTION']
df_inscrit = all_data.drop(columns=list_col_to_drop)
print("✅ Sélection des colonnes importantes dans df_inscrit.")

print("--------------------------------------------------")
print("Informations général du DataFrame :")
print("--------------------------------------------------")
df_inscrit.info()

✅ Sélection des colonnes importantes dans df_inscrit.
--------------------------------------------------
Informations général du DataFrame :
--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1141120 entries, 0 to 1141119
Data columns (total 14 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   NUMERO                 1141120 non-null  object 
 1   SEXE                   1141120 non-null  object 
 2   ANNEE_BACC             1136174 non-null  float64
 3   NATIONALITE            1141120 non-null  object 
 4   SERIE_BACC             1109114 non-null  object 
 5   ETABLISSMENT_CODE      1141120 non-null  object 
 6   NIVEAU_SECTION         1141120 non-null  object 
 7   ANNEE_INSCRIPTION      1141120 non-null  int64  
 8   ANNEE_UNIVERSITAIRE    1141120 non-null  object 
 9   TYPE_FORMATION         1141120 non-null  object 
 10  CODE_NIVEAU            1141120 non-null  i

### Données des Résultats

In [11]:
print("Informations général du DataFrame :")
print("--------------------------------------------------")
df_resultat.info()

Informations général du DataFrame :
--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 753693 entries, 0 to 753692
Data columns (total 27 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   NUMERO                      753693 non-null  object 
 1   NOM                         753693 non-null  object 
 2   PRENOM                      753693 non-null  object 
 3   SEXE                        753693 non-null  object 
 4   NATIONALITE                 753693 non-null  object 
 5   ANNEE UNIVERSITAIRE         753693 non-null  object 
 6   EMAILE INSTITUTIONEL        746075 non-null  object 
 7   EMAIL PERSONNEL             721119 non-null  object 
 8   PORTABLE                    746767 non-null  object 
 9   FORMATION                   753693 non-null  object 
 10  TYPE FORMATION              753562 non-null  object 
 11  DEPARTEMENT                 753562 non-null

#### DataFrame avec seulement les variables les plus importants

#### Remarque :

La plus part des colonnes sont déjà dans le DataFrame des Inscrits donc on les intégrés pas dans le DataFrame des résultats pour la fusion car ils seront en doublons 

In [12]:
df_resultat = df_resultat[['NUMERO', 'ANNEE UNIVERSITAIRE', 'RESULTAT']]
print("✅ Sélection des colonnes importantes dans df_resultat.")
print("--------------------------------------------------")

print("Informations général du DataFrame :")
print("--------------------------------------------------")
df_resultat.info()

✅ Sélection des colonnes importantes dans df_resultat.
--------------------------------------------------
Informations général du DataFrame :
--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 753693 entries, 0 to 753692
Data columns (total 3 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   NUMERO               753693 non-null  object
 1   ANNEE UNIVERSITAIRE  753693 non-null  object
 2   RESULTAT             753562 non-null  object
dtypes: object(3)
memory usage: 17.3+ MB


#### Correction de la syntaxe pour l’homogénéité

In [13]:
df_inscrit.rename(columns={'ANNEE_UNIVERSITAIRE': 'ANNEE UNIVERSITAIRE'}, inplace=True) # Renommer la colonne pour correspondre à df_resultat
df_inscrit['NUMERO'] = df_inscrit['NUMERO'].str.replace(r'^\.', '', regex=True) # Supprimer le point au début
print("✅ Renommage de la colonne ANNEE_UNIVERSITAIRE et suppression du point au début de NUMERO dans df_inscrit.")

✅ Renommage de la colonne ANNEE_UNIVERSITAIRE et suppression du point au début de NUMERO dans df_inscrit.


## Fusion des deux datas frame

### Remarque :

Comme les données couvrent plusieurs années, les variables ***NUMERO*** et ***ANNEE UNIVERSITAIRE*** constituent ensemble la clé primaire pour la fusion.

In [14]:
df_final = pd.merge(df_inscrit, # 
                    df_resultat, # 
                    on=['NUMERO', 'ANNEE UNIVERSITAIRE'], # 
                    how='left' 
                    )
print("✅ Fusion des DataFrames df_inscrit et df_resultat réussie.")

✅ Fusion des DataFrames df_inscrit et df_resultat réussie.


In [15]:
print("Informations général du DataFrame :")
print("--------------------------------------------------")
df_final.info()

Informations général du DataFrame :
--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1215837 entries, 0 to 1215836
Data columns (total 15 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   NUMERO                 1215837 non-null  object 
 1   SEXE                   1215837 non-null  object 
 2   ANNEE_BACC             1210891 non-null  float64
 3   NATIONALITE            1215837 non-null  object 
 4   SERIE_BACC             1183525 non-null  object 
 5   ETABLISSMENT_CODE      1215837 non-null  object 
 6   NIVEAU_SECTION         1215837 non-null  object 
 7   ANNEE_INSCRIPTION      1215837 non-null  int64  
 8   ANNEE UNIVERSITAIRE    1215837 non-null  object 
 9   TYPE_FORMATION         1215837 non-null  object 
 10  CODE_NIVEAU            1215837 non-null  int64  
 11  NIVEAU LMD ET NON LMD  1215837 non-null  object 
 12  SYSTEME                1215837 non-null

In [16]:
print("les années universitaires disponibles :")
print(df_final['ANNEE UNIVERSITAIRE'].unique())

les années universitaires disponibles :
['2010-2011' '2011-2012' '2012-2013' '2013-2014' '2014-2015' '2015-2016'
 '2016-2017' '2017-2018' '2018-2019' '2019-2020' '2020-2021' '2021-2022'
 '2022-2023' '2023-2024']


### Télécharge de data finale pour notre étude

In [ ]:
# df_final.to_csv("df_inscrit_resultat.csv", index=False)